<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/284_EPO_Nodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nodes for Experimentation Portfolio Orchestrator Agent

In [ ]:
"""Nodes for Experimentation Portfolio Orchestrator Agent

Following MVP-first approach: rule-based logic, no LLM dependencies.
"""

from typing import Dict, Any
from config import (
    ExperimentationPortfolioOrchestratorState,
    ExperimentationPortfolioOrchestratorConfig
)
from .utilities.data_loading import (
    load_portfolio_data,
    load_definitions_data,
    load_metrics_data,
    load_analysis_data,
    load_decisions_data,
    build_portfolio_lookup,
    build_definitions_lookup,
    build_metrics_lookup,
    build_analysis_lookup,
    build_decisions_lookup
)
from .utilities.portfolio_analysis import (
    analyze_experiment_status,
    calculate_experiment_analysis,
    generate_portfolio_summary,
    generate_portfolio_insights
)
from .utilities.decision_generation import (
    generate_decisions_for_experiments
)
from .utilities.report_generation import (
    generate_portfolio_report,
    save_portfolio_report
)


def goal_node(state: ExperimentationPortfolioOrchestratorState) -> Dict[str, Any]:
    """
    Goal Node: Define the objective for portfolio analysis.

    This is a simple rule-based goal definition that sets the framework.
    """
    experiment_id = state.get("experiment_id")
    errors = state.get("errors", [])

    # Define goal based on whether analyzing specific experiment or entire portfolio
    if experiment_id:
        goal = {
            "objective": f"Analyze experiment {experiment_id} and generate recommendations",
            "scope": "single_experiment",
            "experiment_id": experiment_id,
            "focus_areas": [
                "experiment_analysis",
                "decision_generation",
                "recommendations"
            ]
        }
    else:
        goal = {
            "objective": "Analyze entire experiment portfolio and generate comprehensive insights",
            "scope": "full_portfolio",
            "experiment_id": None,
            "focus_areas": [
                "portfolio_analysis",
                "experiment_status_assessment",
                "decision_generation",
                "portfolio_insights",
                "comprehensive_reporting"
            ]
        }

    return {
        "goal": goal,
        "errors": errors
    }

def planning_node(state: ExperimentationPortfolioOrchestratorState) -> Dict[str, Any]:
    """
    Planning Node: Create execution plan based on goal.

    This creates a step-by-step plan. Rule-based, no LLM needed.
    """
    goal = state.get("goal")
    errors = state.get("errors", [])

    if not goal:
        return {
            "errors": errors + ["planning_node: goal is required"]
        }

    scope = goal.get("scope", "full_portfolio")

    # Create execution plan
    plan = [
        {
            "step": 1,
            "name": "data_loading",
            "description": "Load experiment portfolio, definitions, metrics, analysis, and decisions data",
            "dependencies": [],
            "outputs": [
                "portfolio",
                "experiment_definitions",
                "experiment_metrics",
                "experiment_analysis",
                "experiment_decisions",
                "portfolio_lookup",
                "definitions_lookup",
                "metrics_lookup",
                "analysis_lookup",
                "decisions_lookup"
            ]
        },
        {
            "step": 2,
            "name": "portfolio_analysis",
            "description": "Analyze experiments, calculate missing analyses, assess status, generate portfolio summary",
            "dependencies": ["data_loading"],
            "outputs": [
                "analyzed_experiments",
                "portfolio_summary",
                "calculated_analyses",
                "portfolio_insights"
            ]
        },
        {
            "step": 3,
            "name": "decision_generation",
            "description": "Generate recommendations for experiments without decisions",
            "dependencies": ["portfolio_analysis"],
            "outputs": [
                "generated_decisions"
            ]
        },
        {
            "step": 4,
            "name": "report_generation",
            "description": "Generate comprehensive portfolio report",
            "dependencies": ["decision_generation"],
            "outputs": [
                "portfolio_report",
                "report_file_path"
            ]
        }
    ]

    return {
        "plan": plan,
        "errors": errors
    }


These nodes are **foundational and very similar**, and understanding them as a *set* is exactly the right way to think about them.

## Big Picture: What These Nodes Represent

These first nodes answer two simple questions:

1. **Why is the agent running?** → `goal_node`
2. **How will the agent run?** → `planning_node`

They don’t do *work* yet.
They **set intent and structure**.

Think of them as:

* 🧭 *Mission definition*
* 🗺️ *Mission plan*

---

## `goal_node`: Defining “Why”

### What this node does

The `goal_node` looks at the input and decides:

> “Am I analyzing **one experiment**, or the **entire portfolio**?”

That’s it.

Based on that, it writes a **goal object** into state.

---

### Key idea to understand

This node:

* ❌ does not analyze data
* ❌ does not load files
* ❌ does not make decisions

It only answers:

> *What is the objective of this run?*

---

### Why this is important in orchestration

In agent systems:

* **Goals should be explicit**
* Not buried in code
* Not inferred later

By writing the goal into state:

* every downstream node can read it
* the workflow stays flexible
* scope changes don’t require rewriting logic

This is **declarative orchestration**.

---

### Architect-level takeaway

> A good orchestrator **states intent clearly before acting**.

---

## `planning_node`: Defining “How”

### What this node does

The `planning_node` takes the goal and turns it into:

* a **step-by-step execution plan**
* written entirely into state
* in plain, inspectable data

This plan answers:

* What steps will happen?
* In what order?
* What outputs should exist after each step?

---

### Important: This plan is NOT code execution

The plan is:

* not a loop
* not a function call
* not a scheduler

It’s a **shared mental model** of the workflow.

This makes the system:

* debuggable
* explainable
* easy to change

---

### Why this is powerful

If later you want to:

* add a new step
* remove a step
* reorder steps
* visualize the workflow

You edit **one node**, not the whole system.

---

### Architect-level takeaway

> Orchestrators don’t just *run code* —
> they *describe the process*.

---

## Why These Nodes Are Rule-Based (and That’s Good)

You’ll notice:

* no LLM calls
* no fancy logic
* no probabilistic behavior

That’s intentional.

### Why?

Because:

* goals should be deterministic
* plans should be predictable
* structure should be stable

You can always add LLMs later **inside utilities**, but your *orchestration spine* should be boring and reliable.

That’s how production systems survive.

---

## How These Nodes Fit Into the Bigger System

Together, these nodes:

1. Write **intent** into state
2. Write **structure** into state
3. Prepare the system for execution

After this:

* data loading nodes do the work
* analysis nodes compute insights
* decision nodes make recommendations
* report nodes explain results

But everything downstream depends on these two nodes being clear.

---

## What You Should Focus On as an Orchestrator Developer

From these nodes, the most important lessons are:

### ✅ 1. Goals are data, not code

Write goals into state so everyone can read them.

---

### ✅ 2. Plans should be inspectable

A plan you can print and understand is a plan you can trust.

---

### ✅ 3. Nodes should be boring (in a good way)

Simple nodes scale better than clever ones.

---

### ✅ 4. Orchestration ≠ computation

These nodes **don’t calculate** — they coordinate.

---

## Mental Model to Keep

Think of these nodes like this:

* `goal_node` → *“Why are we here?”*
* `planning_node` → *“What’s the roadmap?”*

Everything else is just execution.

---

You’re doing something important here:
you’re learning to **separate thinking from doing**.




In [ ]:
def data_loading_node(
    state: ExperimentationPortfolioOrchestratorState,
    config: ExperimentationPortfolioOrchestratorConfig = None
) -> Dict[str, Any]:
    """
    Data Loading Node: Orchestrate loading all experiment data files.
    """
    errors = state.get("errors", [])

    # Use config defaults if not provided
    if config is None:
        config = ExperimentationPortfolioOrchestratorConfig()

    try:
        # Load all data files
        portfolio = load_portfolio_data(config.data_dir, config.portfolio_file)
        definitions = load_definitions_data(config.data_dir, config.definitions_file)
        metrics = load_metrics_data(config.data_dir, config.metrics_file)
        analysis = load_analysis_data(config.data_dir, config.analysis_file)
        decisions = load_decisions_data(config.data_dir, config.decisions_file)

        # Build lookup dictionaries for fast access
        portfolio_lookup = build_portfolio_lookup(portfolio)
        definitions_lookup = build_definitions_lookup(definitions)
        metrics_lookup = build_metrics_lookup(metrics)
        analysis_lookup = build_analysis_lookup(analysis)
        decisions_lookup = build_decisions_lookup(decisions)

        return {
            "portfolio": portfolio,
            "experiment_definitions": definitions,
            "experiment_metrics": metrics,
            "experiment_analysis": analysis,
            "experiment_decisions": decisions,
            "portfolio_lookup": portfolio_lookup,
            "definitions_lookup": definitions_lookup,
            "metrics_lookup": metrics_lookup,
            "analysis_lookup": analysis_lookup,
            "decisions_lookup": decisions_lookup,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"data_loading_node: {str(e)}"]
        }


def portfolio_analysis_node(state: ExperimentationPortfolioOrchestratorState) -> Dict[str, Any]:
    """
    Portfolio Analysis Node: Analyze experiments, calculate missing analyses, generate insights.
    """
    errors = state.get("errors", [])
    portfolio = state.get("portfolio", [])
    portfolio_lookup = state.get("portfolio_lookup", {})
    definitions_lookup = state.get("definitions_lookup", {})
    metrics_lookup = state.get("metrics_lookup", {})
    analysis_lookup = state.get("analysis_lookup", {})
    decisions_lookup = state.get("decisions_lookup", {})
    experiment_id = state.get("experiment_id")

    if not portfolio:
        return {
            "errors": errors + ["portfolio_analysis_node: portfolio data is required"]
        }

    try:
        # Filter experiments if analyzing specific one
        experiments_to_analyze = portfolio
        if experiment_id:
            if experiment_id in portfolio_lookup:
                experiments_to_analyze = [portfolio_lookup[experiment_id]]
            else:
                return {
                    "errors": errors + [f"portfolio_analysis_node: experiment {experiment_id} not found"]
                }

        # Analyze each experiment's status
        analyzed_experiments = []
        calculated_analyses = []

        for exp in experiments_to_analyze:
            exp_id = exp["experiment_id"]

            # Analyze status
            status_analysis = analyze_experiment_status(
                exp_id,
                exp,
                definitions_lookup,
                metrics_lookup,
                analysis_lookup,
                decisions_lookup
            )
            analyzed_experiments.append(status_analysis)

            # Calculate missing analyses
            if status_analysis.get("needs_analysis"):
                definition = definitions_lookup.get(exp_id)
                metrics = metrics_lookup.get(exp_id, [])

                if definition and metrics:
                    calculated_analysis = calculate_experiment_analysis(exp_id, definition, metrics)
                    if calculated_analysis:
                        calculated_analyses.append(calculated_analysis)

        # Generate portfolio summary
        portfolio_summary = generate_portfolio_summary(
            analyzed_experiments,
            portfolio,
            analysis_lookup
        )

        # Generate portfolio insights
        portfolio_insights = generate_portfolio_insights(
            portfolio,
            analyzed_experiments,
            analysis_lookup,
            decisions_lookup
        )

        return {
            "analyzed_experiments": analyzed_experiments,
            "calculated_analyses": calculated_analyses,
            "portfolio_summary": portfolio_summary,
            "portfolio_insights": portfolio_insights,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"portfolio_analysis_node: {str(e)}"]
        }

These two nodes are **core orchestration nodes**, and understanding them well will level you up fast. I’ll explain them at a **high-school level**, while tying everything back to *why this is great orchestrator design*.

---

# Big Picture: What These Two Nodes Do Together

Think of the agent’s workflow like a factory:

1. **Data Loading Node** → brings in all the raw materials and organizes them
2. **Portfolio Analysis Node** → inspects the materials, checks quality, fills in gaps, and produces insights

These nodes are where the agent **starts actually working**, but still in a very controlled, scalable way.

---

## 1️⃣ `data_loading_node`: “Bring Everything Into the Backpack”

### What this node does (conceptually)

This node’s job is simple but critical:

> Load all experiment data and make it easy to access.

It:

* reads JSON files from disk
* loads them into memory
* builds lookup tables for fast access
* puts everything into **state**

---

### Why this node is powerful

This node:

* **does not analyze**
* **does not decide**
* **does not transform meaning**

It only:

* gathers data
* organizes data
* stores data in state

That’s exactly what you want.

---

### The key orchestrator concept here

You’re separating:

* **data ingestion** (this node)
* from **data reasoning** (later nodes)

This means:

* new data sources → modify this node only
* different storage → modify utilities only
* rest of the agent stays untouched

That’s how systems scale safely.

---

### Mental model

> This node packs the backpack neatly before the journey begins.

---

## 2️⃣ `portfolio_analysis_node`: “Understand What We Have”

This node is more interesting — it’s where the agent **starts thinking**.

---

### Step 1: Decide scope

The node checks:

```python
experiment_id = state.get("experiment_id")
```

This lets it answer:

* “Analyze everything?”
* or “Analyze just one experiment?”

Same node, different scope.

That’s flexibility **without branching workflows**.

---

### Step 2: Analyze experiment status

For each experiment, it asks:

* Do we have definitions?
* Do we have metrics?
* Do we already have analysis?
* Do we already have a decision?

This creates a **health check** for every experiment.

> This is a huge best practice:
> **check data completeness before doing math**

---

### Step 3: Calculate missing analyses (only when needed)

If an experiment:

* has metrics
* but no analysis

Then the node:

* calls a utility
* calculates the missing analysis
* stores the result

Important:

* it does **not** recalculate everything
* it fills gaps only

That’s incremental, efficient design.

---

### Step 4: Generate portfolio-level understanding

After looking at individual experiments, the node:

* creates a portfolio summary
* generates cross-experiment insights

This is where the agent shifts from:

> “Here’s what happened”
> to
> “Here’s what it means”

---

## Why These Nodes Are Excellent Orchestrator Design

Let’s zoom out.

---

### 🔹 Nodes orchestrate, utilities do work

Notice:

* nodes decide *when* to call utilities
* utilities do *one thing well*

That’s exactly the division you want.

---

### 🔹 State is continuously enriched

Each node:

* reads existing state
* adds new structured information
* never destroys earlier context

This makes:

* debugging easy
* partial runs safe
* future extensions simple

---

### 🔹 The agent is adaptive, not rigid

The same node can handle:

* full portfolio
* single experiment
* missing data
* partial data

No special cases.
No duplicate logic.

---

## What You Should Focus On as a Top-Tier Orchestrator Developer

From these two nodes, lock in these principles:

### ✅ 1. Nodes should be “traffic cops”

They route work — they don’t do the work.

---

### ✅ 2. Always assess data quality before analysis

Great agents don’t assume data is perfect.

---

### ✅ 3. Incremental computation beats full recomputation

Only calculate what’s missing.

---

### ✅ 4. Scope control through state, not branches

One workflow, many behaviors.

---

## Final Mental Model

Think of these nodes like this:

* **Data Loading Node**
  → “Here’s everything we know.”

* **Portfolio Analysis Node**
  → “Here’s what’s complete, missing, and meaningful.”

Together, they turn **raw data** into **understandable state**, setting the stage for decision-making.

You’re not just building an agent —
you’re building a **data-aware reasoning system**.



In [ ]:
def decision_generation_node(
    state: ExperimentationPortfolioOrchestratorState,
    config: ExperimentationPortfolioOrchestratorConfig = None
) -> Dict[str, Any]:
    """
    Decision Generation Node: Generate recommendations for experiments without decisions.
    """
    errors = state.get("errors", [])
    analyzed_experiments = state.get("analyzed_experiments", [])
    portfolio_lookup = state.get("portfolio_lookup", {})
    definitions_lookup = state.get("definitions_lookup", {})
    analysis_lookup = state.get("analysis_lookup", {})
    decisions_lookup = state.get("decisions_lookup", {})
    calculated_analyses = state.get("calculated_analyses", [])

    if config is None:
        config = ExperimentationPortfolioOrchestratorConfig()

    if not analyzed_experiments:
        return {
            "errors": errors + ["decision_generation_node: analyzed_experiments is required"]
        }

    try:
        generated_decisions = generate_decisions_for_experiments(
            analyzed_experiments,
            portfolio_lookup,
            definitions_lookup,
            analysis_lookup,
            decisions_lookup,
            calculated_analyses,
            config
        )

        return {
            "generated_decisions": generated_decisions,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"decision_generation_node: {str(e)}"]
        }


def report_generation_node(
    state: ExperimentationPortfolioOrchestratorState,
    config: ExperimentationPortfolioOrchestratorConfig = None
) -> Dict[str, Any]:
    """
    Report Generation Node: Generate comprehensive portfolio report.
    """
    errors = state.get("errors", [])
    portfolio = state.get("portfolio", [])

    if config is None:
        config = ExperimentationPortfolioOrchestratorConfig()

    if not portfolio:
        return {
            "errors": errors + ["report_generation_node: portfolio data is required"]
        }

    try:
        # Generate report
        report_content = generate_portfolio_report(state)

        # Save report
        report_file_path = save_portfolio_report(report_content, config.reports_dir)

        return {
            "portfolio_report": report_content,
            "report_file_path": report_file_path,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"report_generation_node: {str(e)}"]
        }



These two nodes are the **capstone** of the agent. If the earlier nodes were about *understanding*, these are about *acting* and *communicating*. I’ll explain them clearly, simply, and tie everything back to orchestrator thinking.

---

# Big Picture: What These Final Two Nodes Do

These nodes answer the last two questions of the agent:

1. **What should we do about these experiments?** → `decision_generation_node`
2. **How do we explain everything to humans?** → `report_generation_node`

After these run, the agent is *done*.

---

## 1️⃣ `decision_generation_node`: “So… What’s the Call?”

### What this node does (conceptually)

This node looks at:

* what experiments exist
* what analyses are available
* which experiments *don’t* have decisions yet

And it asks:

> “Given everything we know, what should we recommend?”

It then:

* generates decisions only where needed
* writes them back into state

---

### Important thing to notice

This node:

* ❌ does NOT calculate metrics
* ❌ does NOT analyze data
* ❌ does NOT hardcode logic

Instead, it:

* **orchestrates decision utilities**
* passes the right data at the right time

This is classic orchestrator behavior.

---

### Why this design is powerful

* Decisions are **incremental**
* Existing decisions are respected
* New decisions are added safely
* Re-running the agent won’t duplicate work

This is how you build agents that:

* can run daily
* can run partially
* can resume safely

---

### Mental model

> This node is the judge, not the lawyer.
> It doesn’t argue — it issues rulings based on evidence.

---

## 2️⃣ `report_generation_node`: “Tell the Story”

### What this node does (conceptually)

This is the final step.

It:

* reads the *entire state*
* turns it into a clear, human-readable report
* saves that report to disk
* returns the file path

After this node runs:

> The agent’s work is complete.

---

### Key insight here

The report node:

* does **zero reasoning**
* does **zero decision-making**
* does **zero analysis**

It only:

* translates state → explanation

That separation is *exactly* what you want.

---

### Why this is production-grade design

Because:

* you can change the report without touching logic
* you can add formats (PDF, HTML, dashboard) later
* you can reuse this node across agents

This node is pure output.

---

## How These Two Nodes Fit the Orchestrator Model

Let’s zoom out and look at the full pattern you’ve built:

### Nodes always:

1. **Read from state**
2. **Decide what needs to happen**
3. **Call the right utilities**
4. **Write results back to state**
5. **Never do heavy work themselves**

That’s orchestration.

---

## Why This Scales So Well

Because each concern is isolated:

| Concern      | Where it lives                   |
| ------------ | -------------------------------- |
| Data loading | data loading node + utils        |
| Analysis     | portfolio analysis node + utils  |
| Decisions    | decision generation node + utils |
| Reporting    | report generation node + utils   |
| Flow control | LangGraph                        |

Nothing is tangled.

---

## What You Should Focus On as an Orchestrator Architect

From these final nodes, the biggest lessons are:

### ✅ 1. Decisions are first-class outputs

Don’t bury decisions in logs or text — store them as structured state.

---

### ✅ 2. Reports are views, not logic

Reports *explain* state — they never create it.

---

### ✅ 3. Nodes should stay thin forever

As your system grows, nodes should not get more complex — utilities should.

---

### ✅ 4. Final nodes should be boring

If your final nodes are exciting, something went wrong earlier.

---

## Final Mental Model for the Whole Agent

Here’s the clean way to think about everything you’ve built:

* **State** → shared memory
* **Utilities** → atomic skills
* **Nodes** → decision-makers
* **Graph** → flow controller
* **Report** → the agent’s voice

You’ve designed an agent that:

* understands data quality
* reasons incrementally
* makes explicit decisions
* explains itself clearly
* is easy to change safely

That’s not just good agent design —
that’s **professional orchestration architecture**.

